In [1]:
from datetime import datetime
import numpy
import matplotlib.pyplot as plt
import collections
import pandas as pd
import numpy as np
import requests
import time
import os

In [2]:
#Get the path reference to read teh data

try:
    __thisdir__ = os.path.dirname(os.path.abspath(__file__))
except NameError as error:
    __thisdir__ = "."
    
#create the direcotry to store the data if not existed
review_data = os.path.join(__thisdir__,"..","beer_reviews.csv")
styles_data = os.path.join(__thisdir__,"..","users","jpastell","data","beer_styles.csv")


In [3]:
#Reading the data
review_df = pd.read_csv(review_data)
style_df = pd.read_csv(styles_data)

In [4]:
#Check the data from the reviews
review_df.head()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883


In [5]:
#Check the data from the reviews
style_df.head()

,id,name,beer_style,ibu_Min,ibu_Max,abv_Min,abv_Max,srm_Min,srm_Max,og_Min,fg_Min,fg_Max
0,0,Classic English-Style Pale Ale,English Pale,20,40,4.5,5.5,5,5,1.040,1.008,1.016
1,1,English-Style India Pale Ale,English IPA,35,63,5.0,7.0,6,14,1.050,1.012,1.018
2,2,Ordinary Bitter,Bitter,20,35,3.0,4.1,5,12,1.033,1.006,1.012
3,3,Special Bitter or Best Bitter,Special Bitter,28,40,4.1,4.8,6,14,1.038,1.006,1.012
4,4,Extra Special Bitter,ESB,30,45,4.8,5.8,8,14,1.046,1.010,1.016


In [6]:
#Normilize the data
review_style_done=list()

#Key for new entries
new_key = style_list = list()

#We need to iterate all beer styles find all matches in teh review and normilize the data
for style in style_df["beer_style"]:
        style_list.append(style)
        #Find all dat athat matches the style defined
        matches_df = pd.DataFrame(review_df["beer_style"]\
                                 [review_df.beer_style.str.contains(style, regex= True, na=False)])
        #Remove the duplicates
        matches_df.drop_duplicates(inplace=True)
        #Iterate all matches and replace the data
        for match in matches_df["beer_style"]:
            #Replace the data if not updated yet and mask it as updated
            if match not in review_style_done:
                print(f"{match} : {style}")
                if match != style:
                    review_df.replace({'beer_style':{match:style}},inplace=True)
                review_style_done.append(match)

English Pale Ale : English Pale
English Pale Mild Ale : English Pale
English Bitter : Bitter
Extra Special / Strong Bitter (ESB) : Bitter
English Dark Mild Ale : English Dark Mild
English Brown Ale : English Brown
Old Ale : Old Ale
English Strong Ale : Strong Ale
American Strong Ale : Strong Ale
Scotch Ale / Wee Heavy : Scotch Ale
Milk / Sweet Stout : Sweet Stout
Oatmeal Stout : Oatmeal Stout
Irish Red Ale : Irish Red
Foreign / Export Stout : Export Stout
American Pale Lager : American Pale
American Pale Ale (APA) : American Pale
American Pale Wheat Ale : American Pale
American IPA : American IPA
American Double / Imperial IPA : Imperial IPA
American Amber / Red Ale : Amber
American Amber / Red Lager : Amber
American Barleywine : American Barleywine
Wheatwine : Wheatwine
American Blonde Ale : Blonde
American Brown Ale : American Brown
American Black Ale : Black Ale
American Stout : American Stout
Export Stout : Stout
Russian Imperial Stout : Stout
American Double / Imperial Stout : Sto

In [20]:
#--------------------------------------#
#            Beer Styles Table         #
#--------------------------------------#

#Next key for styles
key_id = len(style_df)

#To creat enew entries
#createing list for each column,
#this is faster than append rows to teh data frame
id = list()
name = list()
short_name = list()
ibuMin = list()
ibuMax = list()
abvMin = list()
abvMax = list()
srmMin = list()
srmMax = list()
ogMin = list()
fgMin = list()
fgMax = list()

#After data normalization get all the styles witouth record and create a new entry
normal_styles_df = pd.DataFrame(review_df["beer_style"])
#Remove the duplicates
normal_styles_df.drop_duplicates(inplace=True)
normal_styles_df

#Get all data syles not defined yet
for style in normal_styles_df["beer_style"]:
    if style not in style_list:
        #not_defined.append(style)
        name.append(style)
        short_name.append(style)
        id.append(key_id)
        #Since this styles are not defined set them to na
        ibuMin.append(np.NaN)
        ibuMax.append(np.NaN)
        abvMin.append(np.NaN)
        abvMax.append(np.NaN)
        srmMin.append(np.NaN)
        srmMax.append(np.NaN)
        ogMin.append(np.NaN)
        fgMin.append(np.NaN)
        fgMax.append(np.NaN)
        key_id+=1

#Build the new data frame
ndef_df = pd.DataFrame({"id":id,
                        "name":name,
                        "beer_style":short_name,
                        "ibu_Min":ibuMin,
                        "ibu_Max":ibuMax,
                        "abv_Min":abvMin,
                        "abv_Max":abvMax,
                        "srm_Min":srmMin,
                        "srm_Max":srmMax,
                        "og_Min":ogMin,
                        "fg_Min":fgMin,
                        "fg_Max":fgMax})

#Join the data
beer_styles_df = pd.concat([style_df, ndef_df])
beer_styles_df.rename(columns={"id":"beer_style_id"},inplace=True)
beer_styles_df.head(50)

,beer_style_id,name,beer_style,ibu_Min,ibu_Max,abv_Min,abv_Max,srm_Min,srm_Max,og_Min,fg_Min,fg_Max
0,0,Classic English-Style Pale Ale,English Pale,20.0,40.0,4.5,5.5,5.0,5.0,1.040,1.008,1.016
1,1,English-Style India Pale Ale,English IPA,35.0,63.0,5.0,7.0,6.0,14.0,1.050,1.012,1.018
2,2,Ordinary Bitter,Bitter,20.0,35.0,3.0,4.1,5.0,12.0,1.033,1.006,1.012
3,3,Special Bitter or Best Bitter,Special Bitter,28.0,40.0,4.1,4.8,6.0,14.0,1.038,1.006,1.012
4,4,Extra Special Bitter,ESB,30.0,45.0,4.8,5.8,8.0,14.0,1.046,1.010,1.016
5,5,English-Style Summer Ale,English Summer Ale,20.0,30.0,3.6,5.0,4.0,7.0,1.036,1.006,1.012
6,6,Scottish-Style Light Ale,Scottish Light,9.0,20.0,2.8,3.5,8.0,17.0,1.030,1.006,1.012
7,7,Scottish-Style Heavy Ale,Scottish Heavy,12.0,20.0,3.5,4.0,10.0,19.0,1.035,1.010,1.014
8,8,Scottish-Style Export Ale,Scottish Export,15.0,25.0,4.0,5.3,10.0,19.0,1.040,1.010,1.018
9,9,English-Style Pale Mild Ale,English Pale Mild,10.0,20.0,3.2,4.0,8.0,17.0,1.030,1.004,1.008


In [8]:
#--------------------------------------#
#   Get a dataframe for all user ID's  #
#--------------------------------------#
#Retrieve the users form the main DF
user_table_df = pd.DataFrame(review_df["review_profilename"])
#Remove duplicates
user_table_df.drop_duplicates(inplace=True)
#Rename the columns to match sql
user_table_df.rename(columns={"review_profilename":"profile_name"},inplace=True)
#Assign the user ID
user_table_df["profile_id"]=numpy.arange(0, len(user_table_df))
#Verify the results
user_table_df.head()

,profile_name,profile_id
0,stcules,0
4,johnmichaelsen,1
5,oline73,2
6,Reidrover,3
7,alpinebryant,4


In [9]:
#--------------------------------------#
#            Brewery  table            #
#--------------------------------------#
brewery_df = pd.DataFrame(review_df[["brewery_id","brewery_name"]])
brewery_df.drop_duplicates(inplace=True)
brewery_df.head()

,brewery_id,brewery_name
0,10325,Vecchio Birraio
4,1075,Caldera Brewing Company
10,163,Amstel Brouwerij B. V.
393,1454,Broad Ripple Brew Pub
403,850,Moon River Brewing Company


In [12]:
#--------------------------------------#
#              Beer table              #
#--------------------------------------#
#Dictionary for mapping
name_style_translate = dict()
#Lis for new column
style_id_entry = list()

beer_table_df = pd.DataFrame(review_df[["beer_beerid","beer_name","brewery_id","beer_abv","beer_style"]])
beer_table_df.drop_duplicates(inplace=True)
beer_table_df.rename(columns={"beer_beerid":"beer_id"},inplace=True)
#beer_table_df.head()

#For performance reasons build a dictionary to map the name and the ID, and build a new column,
#This is faster than dataframe replace instruction

#Dictionary creation
for row in beer_styles_df[["beer_style","beer_style_id"]].iterrows():
    name_style_translate.update({row[1]["beer_style"]:row[1]["beer_style_id"]})
    
#build the column for the user id
for beer in beer_table_df["beer_style"]:
    style_id_entry.append(name_style_translate[beer])
    
beer_table_df["beer_style_id"]=style_id_entry
beer_table_df.head()

,beer_id,beer_name,brewery_id,beer_abv,beer_style,beer_style_id
0,47986,Sausa Weizen,10325,5.0,Hefeweizen,42
1,48213,Red Moon,10325,6.2,Strong Ale,13
2,48215,Black Horse Black Beer,10325,6.5,Stout,38
3,47969,Sausa Pils,10325,5.0,German Pilsener,66
4,64883,Cauldron DIPA,1075,7.7,Imperial IPA,27


In [16]:
#--------------------------------------#
#              Review table            #
#--------------------------------------#
beer_reviews_df = pd.DataFrame(review_df[["review_profilename",\
                                          "review_overall",\
                                          "review_aroma",\
                                          "review_appearance",\
                                          "review_palate",\
                                          "review_taste",\
                                          "beer_beerid"]])


In [14]:
#For performance reasons build a dictionary to map the name and the ID, and build a new column,
#This is faster than dataframe replace instruction

#Dictionary for mapping
name_translate = dict()
#Lis for new column
user_id_entry = list()

#Dictionary creation
for row in user_table_df.iterrows():
    name_translate.update({row[1]["profile_name"]:row[1]["profile_id"]})
    
#build the column for the user id
for beer in beer_reviews_df["review_profilename"]:
    user_id_entry.append(name_translate[beer])

In [15]:
time_column = list()

#Conver the time in the table
for row in review_df[["review_time"]].iterrows():
    time_column.append(datetime.utcfromtimestamp(row[1]["review_time"]))

In [17]:
#Insert the missing colmuns
beer_reviews_df["review_id"]=numpy.arange(0, len(beer_reviews_df))
beer_reviews_df["user_profile_id"]=user_id_entry
beer_reviews_df["review_time"]=time_column

In [18]:
#Rename columns with diferent names and drop the data we don't need
beer_reviews_df.drop(["review_profilename"],axis=1,inplace=True)
beer_reviews_df.rename(columns={"beer_beerid":"beer_id"},inplace=True)

In [19]:
beer_reviews_df.head()

,review_overall,review_aroma,review_appearance,review_palate,review_taste,beer_id,review_id,user_profile_id,review_time
0,1.5,2.0,2.5,1.5,1.5,47986,0,0,2009-02-16 20:57:03
1,3.0,2.5,3.0,3.0,3.0,48213,1,0,2009-03-01 13:44:57
2,3.0,2.5,3.0,3.0,3.0,48215,2,0,2009-03-01 14:10:04
3,3.0,3.0,3.5,2.5,3.0,47969,3,0,2009-02-15 19:12:25
4,4.0,4.5,4.0,4.0,4.5,64883,4,1,2010-12-30 18:53:26
